In [2]:
from pymongo import MongoClient
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
from functools import reduce
import operator
import json
import re

In [102]:
df = pd.read_csv('df_final_clean3.csv', index_col=0)
df.dropna(how = "all", axis = 0, inplace = True)

In [103]:
df.head()

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3
indice,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España"
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España"
95.0,Casa Labra,"Calle Tetuan 12, 28013 Madrid España",Bar,#650 de 10.414 restaurantes en Madrid,€,Calle Tetuan 12,Madrid España,Calle Tetuan 12,"Calle Tetuan 12, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 12, Madrid España"
134.0,Parrilla El Gaucho,"Calle Tetuan 34, 28013 Madrid España",Restaurante de carne,#311 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Tetuan 34,Madrid España,Calle Tetuan 34,"Calle Tetuan 34, Madrid España","(40.0395885, -3.6142713)","Calle Tetuan 34, Madrid España"
279.0,Santa Eulalia Boulangerie,"Calle Espejo 12, 28013 Madrid España",Confiterías/panaderías,N.º 23 de 477 cafeterías y teterías en Madrid,€€ - €€€,Calle Espejo 12,Madrid España,Calle Espejo 12,"Calle Espejo 12, Madrid España","(40.1384846, -3.4252641)","Calle Espejo 12, Madrid España"


In [104]:
import ast

In [105]:
def ast_literal (x):
    try:
        return ast.literal_eval(x)
    except:
        return x

In [106]:
df.coordinates = df.coordinates.apply(ast_literal)

In [107]:
type(df.coordinates[0])

tuple

In [108]:
df.dtypes

name           object
address        object
type           object
ranking        object
price          object
direccion      object
comunidad      object
street         object
street_2       object
coordinates    object
street_3       object
dtype: object

In [109]:
def type_point(x):
    return {"type":"Point", "coordinates": (x[1], x[0])}
 

In [110]:
df["type_point"] = df["coordinates"].apply(type_point)

In [111]:
df.head(2)

,name,address,type,ranking,price,direccion,comunidad,street,street_2,coordinates,street_3,type_point
indice,,,,,,,,,,,,
413.0,El anciano rey de los vinos,"Calle Bailén 19, 28013 Madrid España",Mediterránea,#549 de 10.409 restaurantes en Madrid,€€ - €€€,Calle Bailén 19,Madrid España,Calle Bailén 19,"Calle Bailén 19, Madrid España","(40.028851, -3.606505)","Calle Bailén 19, Madrid España","{'type': 'Point', 'coordinates': (-3.606505, 4..."
76.0,Taberna Más Al Sur,"Calle Santa Isabel Nº 35, 28012 Madrid España",Mediterránea,#34 de 10.414 restaurantes en Madrid,€€ - €€€,Calle Santa Isabel Nº 35,Madrid España,Calle Santa Isabel 35,"Calle Santa Isabel 35, Madrid España","(40.0316837, -3.5908706)","Calle Santa Isabel 35, Madrid España","{'type': 'Point', 'coordinates': (-3.5908706, ..."


In [112]:
#dataframe.to_json(orient = 'index')

## Exportar a Mongo

In [113]:
client = MongoClient("localhost:27017")

In [114]:
db = client.get_database("PROYECTO")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'PROYECTO')

In [115]:
collection = db["Restaurantes"]

In [116]:
collection.create_index([("type_point_", "2dsphere")])

'type_point__2dsphere'

In [117]:
data = df.to_dict(orient='records')

In [119]:
collection.insert_many(data)

1- Input preguntando donde estas
2- Input preguntando que radio quieren
3- Funcion que pasa de direccion a coordenadas (coordenadas al reves)
4- Funcion que me haga una query near a moongo y me devuelva todos los restaurantes en el radio  